In [13]:
# Подключаем библиотеку для работы с нейронными сетями
import keras

In [14]:
# Настройки нейронной сети
# Колличество примеров за один проход
batch_size = 128
# Колличество классов
num_classes = 10
# Сколько раз нейронная сеть увидит весь датасет 
epochs = 20

In [15]:
# Из модуля с наборами данных берем MNIST
from keras.datasets import mnist

# Данные уже перемешанны и разбиты на обубчающую и тестовые выборки
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [16]:
# У нас сейчас выборка представленна в виде 60 тысяч картинок 28 на 28 пикселей - т.е. массив 2-х мерных массивов
x_train.shape

(60000, 28, 28)

In [17]:
# С помощью метода numpy reshape мы можем развернуть картинки в одномерный массив
x_train.reshape(60000, 784).shape

(60000, 784)

In [18]:
# Разворачиваем обе выборки
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

In [19]:
# По умолчанию тип чисел - unit8, но наша библиотека работает с числами типа float32
x_train[0][0].dtype

dtype('uint8')

In [20]:
# Поэтому мы конвертируем тип
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [21]:
# Сейчас пиксили картинки представленный в виде чисел от 0 до 255
# Мы хотим подовать нейронной сети числа от 0 до 1 поэтому мы их нормируем
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [22]:
# Преобразуем номера классов в one-hot вектора
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [23]:
# Выбираем один из вариантов как мы хотим описывать нейронную сеть - в виде послеждовательности слоев.
from keras.models import Sequential
# Выбираем два слоя Обычный полносвязный перцептрон и слой Dropout - для регуляризации
from keras.layers import Dense, Dropout
# Выбираем оптимизатор - метод с помощью которого мы будем обновлять веса нашей нейронной сети 
from keras.optimizers import RMSprop

model = Sequential()
# Входный слой - 784 (24 * 24) нейронов 
# Первый скрытый слой 512 нейроннов, ф-я активации Relu, 
model.add(Dense(512, activation='relu', input_shape=(784,)))
# Dropout слой, каждый прогон отключает 20 процентов связей
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
# Выходной слой на 10 нейронов, 
model.add(Dense(num_classes, activation='softmax'))
# Выводим архитектуру сети
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Компилируем сеть, задав функцию ошибки - Кросс энтропия для задачи классификации, оптимизатор RMSprop, и метрика accuracy
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [25]:
# Настраиваем интерактивную систему логов
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Task1_tb_logs', histogram_freq=0, write_graph=True, write_images=True)
# Тренеруем нейронную сеть, задав все объявленные выше параметры
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test), callbacks=[tbCallBack])

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 4s - loss: 0.2463 - acc: 0.9245 - val_loss: 0.1056 - val_acc: 0.9667
Epoch 2/20
60000/60000 [==============================] - 2s - loss: 0.1036 - acc: 0.9688 - val_loss: 0.0793 - val_acc: 0.9749
Epoch 3/20
60000/60000 [==============================] - 2s - loss: 0.0750 - acc: 0.9773 - val_loss: 0.0725 - val_acc: 0.9783
Epoch 4/20
60000/60000 [==============================] - 2s - loss: 0.0600 - acc: 0.9817 - val_loss: 0.0767 - val_acc: 0.9790
Epoch 5/20
60000/60000 [==============================] - 2s - loss: 0.0507 - acc: 0.9845 - val_loss: 0.0725 - val_acc: 0.9822
Epoch 6/20
60000/60000 [==============================] - 2s - loss: 0.0422 - acc: 0.9866 - val_loss: 0.0866 - val_acc: 0.9784
Epoch 7/20
60000/60000 [==============================] - 2s - loss: 0.0386 - acc: 0.9887 - val_loss: 0.0817 - val_acc: 0.9825
Epoch 8/20
60000/60000 [==============================] - 2s 

In [26]:
# Выполняем и смотрим результат
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.123294936596
Test accuracy: 0.9832
